In [2]:
from pyspark.ml.feature import PolynomialExpansion
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import PolynomialExpansion
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col

In [3]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("GAM") \
    .getOrCreate()

### Loading Data

In [4]:
# Load dataset 
train_path = "gs://msca-bdp-student-gcs/Group1/lr_data/traindata_transformed/"
df_train = spark.read.parquet(train_path, header=True, inferSchema=True)

df_train.show(10)

+--------------------+----------+----------+--------------+-----------+------------+---------+--------------+----------------+--------+--------+-----------------+------------+---------------+-------------+-----------------+-------------------+----------------------+-------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+
|               legId|searchDate|flightDate|travelDuration|elapsedDays|isRefundable|totalFare|seatsRemaining|DaysBeforeFlight|Layovers|NumStops|NumUniqueAirlines|AircraftType|NumUniqueCabins|hasFirstClass|FlightArrivalDate|totalTravelDistance|startingAirport_onehot|destinationAirport_onehot|AirlineName_0_onehot|AirlineName_1_onehot|AirlineName_2_onehot|AirlineName_3_onehot|AirlineName_4_onehot|    log_totalFare|
+--------------------+----------+----------+--------------+-----------+------------+---------+--------------+----------------+--------+--------+-----------------+------

In [5]:
# Load dataset 
test_path = "gs://msca-bdp-student-gcs/Group1/lr_data/testdata_transformed/"
df_test = spark.read.parquet(train_path, header=True, inferSchema=True)

df_test.show(10)

+--------------------+----------+----------+--------------+-----------+------------+---------+--------------+----------------+--------+--------+-----------------+------------+---------------+-------------+-----------------+-------------------+----------------------+-------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+
|               legId|searchDate|flightDate|travelDuration|elapsedDays|isRefundable|totalFare|seatsRemaining|DaysBeforeFlight|Layovers|NumStops|NumUniqueAirlines|AircraftType|NumUniqueCabins|hasFirstClass|FlightArrivalDate|totalTravelDistance|startingAirport_onehot|destinationAirport_onehot|AirlineName_0_onehot|AirlineName_1_onehot|AirlineName_2_onehot|AirlineName_3_onehot|AirlineName_4_onehot|    log_totalFare|
+--------------------+----------+----------+--------------+-----------+------------+---------+--------------+----------------+--------+--------+-----------------+------

#### Repartitioning Data

In [6]:
# Repartition train data
num = df_train.rdd.getNumPartitions()
print(num)
df_train = df_train.repartition(num)

20


In [7]:
# Repartition test data
num = df_test.rdd.getNumPartitions()
print(num)
df_test = df_test.repartition(num)

20


### Baseline Model

In [27]:
# Assemble features into a single vector column

feature_columns = [
    "DaysBeforeFlight", "NumStops", "NumUniqueAirlines", "NumUniqueCabins", 
    "travelDuration", "isRefundable", "hasFirstClass", "totalTravelDistance",
    "AircraftType", "seatsRemaining", "elapsedDays"
]

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df = assembler.transform(df_train)

# Apply polynomial expansion to degree 4
poly_expansion = PolynomialExpansion(inputCol="features", outputCol="poly_features", degree=4)

df = poly_expansion.transform(df)

df.select("features", "poly_features").show(5)


+--------------------+--------------------+
|            features|       poly_features|
+--------------------+--------------------+
|[3.0,1.0,1.0,1.0,...|[3.0,9.0,27.0,81....|
|[45.0,1.0,1.0,1.0...|[45.0,2025.0,9112...|
|[20.0,1.0,1.0,1.0...|[20.0,400.0,8000....|
|[55.0,1.0,1.0,1.0...|[55.0,3025.0,1663...|
|[57.0,0.0,1.0,1.0...|[57.0,3249.0,1851...|
+--------------------+--------------------+
only showing top 5 rows



In [28]:
# Prepare the data for GLM
df2 = df.withColumn("target", df["log_totalFare"])  
df2 = df2.drop("features")
df2 = df2.withColumnRenamed("poly_features", "features") 

# Create and fit the GLM model (using Gaussian family and identity link)
glm = GeneralizedLinearRegression(featuresCol="features", labelCol="target", family="gaussian", link="identity")

# Fit the model
model = glm.fit(df2)

24/12/02 02:45:13 WARN org.apache.spark.ml.util.Instrumentation: [fb4683da] regParam is zero, which might cause numerical instability and overfitting.
24/12/02 03:12:55 WARN org.apache.spark.ml.util.Instrumentation: [fb4683da] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [29]:
# Make predictions
predictions = model.transform(df2)

# Initialize the evaluator to compute R2
evaluator = RegressionEvaluator(labelCol="target", predictionCol="prediction", metricName="r2")

# Calculate R²
r2 = evaluator.evaluate(predictions)
print("R^2: ", r2)

R^2:  0.43175539395783724


In [30]:
# Initialize the evaluator to compute RMSE
evaluator = RegressionEvaluator(labelCol="target", predictionCol="prediction", metricName="rmse")

# Calculate RMSE
rmse = evaluator.evaluate(predictions)
print("RMSE: ", rmse)

RMSE:  0.43585079012796013


### Model with Backwards Elimination Features

In [26]:
from pyspark.sql.functions import col

# Create interaction term between number
df_train = df_train.withColumn("NumUniqueCabins_hasFirstClass", col("NumUniqueCabins") * col("hasFirstClass"))

# Update feature_columns to include interaction terms
feature_columns = [
    'travelDuration','elapsedDays', 'isRefundable', 'seatsRemaining', 'DaysBeforeFlight', 
    'NumStops', 'NumUniqueAirlines', 'AircraftType', 'NumUniqueCabins', 
    'hasFirstClass','startingAirport_onehot', 'destinationAirport_onehot',
    'AirlineName_0_onehot', 'NumUniqueCabins_hasFirstClass'
]

In [27]:
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df = assembler.transform(df_train)

# Apply polynomial expansion with degre 4
poly_expansion = PolynomialExpansion(inputCol="features", outputCol="poly_features", degree=2)

df = poly_expansion.transform(df)

# Show the transformed features
df.select("features", "poly_features").show(5)

+--------------------+--------------------+
|            features|       poly_features|
+--------------------+--------------------+
|(54,[0,3,4,5,6,7,...|(1539,[0,1,9,10,1...|
|(54,[0,3,4,5,6,7,...|(1539,[0,1,9,10,1...|
|(54,[0,3,4,5,6,7,...|(1539,[0,1,9,10,1...|
|(54,[0,3,4,5,6,7,...|(1539,[0,1,9,10,1...|
|(54,[0,3,4,6,7,8,...|(1539,[0,1,9,10,1...|
+--------------------+--------------------+
only showing top 5 rows



In [28]:
# Prepare the data for GLM
df2 = df.withColumn("target", df["log_totalFare"])  
df2 = df2.drop("features")
df2 = df2.withColumnRenamed("poly_features", "features")  

# Create and fit the GLM model (using Gaussian family and identity link)
glm = GeneralizedLinearRegression(featuresCol="features", labelCol="target", family="gaussian", link="identity")

# Fit the model
model = glm.fit(df2)

24/12/02 18:56:07 WARN org.apache.spark.ml.util.Instrumentation: [0f252abe] regParam is zero, which might cause numerical instability and overfitting.
24/12/02 18:59:22 WARN org.apache.spark.ml.util.Instrumentation: [0f252abe] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [29]:
# Make predictions
predictions = model.transform(df2)

# Initialize the evaluator to compute R2
evaluator = RegressionEvaluator(labelCol="target", predictionCol="prediction", metricName="r2")

# Calculate R² 
r2 = evaluator.evaluate(predictions)
print("R^2: ", r2)

R^2:  0.576656509774735


In [ ]:
# Initialize the evaluator to compute RMSE
evaluator = RegressionEvaluator(labelCol="target", predictionCol="prediction", metricName="rmse")

# Calculate RMSE
rmse = evaluator.evaluate(predictions)
print("RMSE: ", rmse)

RMSE:  0.37619811892184796


In [ ]:
# Create interaction terms
df_test = df_test.withColumn("NumUniqueCabins_hasFirstClass", col("NumUniqueCabins") * col("hasFirstClass"))

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df = assembler.transform(df_test)

# Apply polynomial expansion with degree 2
poly_expansion = PolynomialExpansion(inputCol="features", outputCol="poly_features", degree=2)

df = poly_expansion.transform(df)

# Prepare the data for GLM
df2 = df.withColumn("target", df["log_totalFare"])  
df2 = df2.drop("features")
df2 = df2.withColumnRenamed("poly_features", "features")  

In [ ]:
# Make predictions
predictions = model.transform(df2)

# Initialize the evaluator to compute RMSE
evaluator = RegressionEvaluator(labelCol="target", predictionCol="prediction", metricName="rmse")

# Calculate RMSE
rmse = evaluator.evaluate(predictions)
print("Test RMSE: ", rmse)

Test RMSE:  0.37619811892184796


### Model with Forward Selection Features

In [15]:
# Assemble features into a single vector column
feature_columns = ["totalTravelDistance","NumUniqueAirlines","startingAirport_onehot", 
                   'AirlineName_0_onehot']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df = assembler.transform(df_train)


poly_expansion = PolynomialExpansion(inputCol="features", outputCol="poly_features", degree=2)

df = poly_expansion.transform(df)

# Show the transformed features
df.select("features", "poly_features").show(5)


+--------------------+--------------------+
|            features|       poly_features|
+--------------------+--------------------+
|(30,[0,1,13,18],[...|(495,[0,1,2,3,4,1...|
|(30,[0,1,12,17],[...|(495,[0,1,2,3,4,9...|
|(30,[0,1,12,17],[...|(495,[0,1,2,3,4,9...|
|(30,[0,1,2,19],[2...|(495,[0,1,2,3,4,5...|
|(30,[0,1,9,17],[7...|(495,[0,1,2,3,4,5...|
+--------------------+--------------------+
only showing top 5 rows



In [16]:
# Prepare the data for GLM
df2 = df.withColumn("target", df["log_totalFare"])  
df2 = df2.drop("features")
df2 = df2.withColumnRenamed("poly_features", "features") 

# Create and fit the GLM model 
glm = GeneralizedLinearRegression(featuresCol="features", labelCol="target", family="gaussian", link="identity")

# Fit the model
model = glm.fit(df2)

24/12/02 18:31:40 WARN org.apache.spark.ml.util.Instrumentation: [c27ee27e] regParam is zero, which might cause numerical instability and overfitting.
24/12/02 18:32:28 WARN org.apache.spark.ml.util.Instrumentation: [c27ee27e] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [17]:
from pyspark.ml.evaluation import RegressionEvaluator

# Make predictions
predictions = model.transform(df2)

# Initialize the evaluator
evaluator = RegressionEvaluator(labelCol="target", predictionCol="prediction", metricName="r2")

# Calculate R² manually
r2 = evaluator.evaluate(predictions)
print("R^2: ", r2)

R^2:  0.4578508208764329


In [18]:
# Initialize the evaluator to compute RMSE
evaluator = RegressionEvaluator(labelCol="target", predictionCol="prediction", metricName="rmse")

# Calculate RMSE
rmse = evaluator.evaluate(predictions)
print("RMSE: ", rmse)

RMSE:  0.4257254163841886


#### Perfomance testing with test df

In [19]:
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df = assembler.transform(df_test)

# Apply polynomial expansion 
poly_expansion = PolynomialExpansion(inputCol="features", outputCol="poly_features", degree=2)

df = poly_expansion.transform(df)

# Prepare the data for GLM
df2 = df.withColumn("target", df["log_totalFare"])  
df2 = df2.drop("features")
df2 = df2.withColumnRenamed("poly_features", "features")  

In [20]:
# Make predictions
predictions = model.transform(df2)

# Initialize the evaluator to compute RMSE
evaluator = RegressionEvaluator(labelCol="target", predictionCol="prediction", metricName="rmse")

# Calculate RMSE
rmse = evaluator.evaluate(predictions)
print("Test RMSE: ", rmse)

Test RMSE:  0.42572541638418865
